In [ ]:
import aiohttp
import asyncio
import urllib.parse
import xml.etree.ElementTree as ET
import json
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

# nest_asyncio 적용
nest_asyncio.apply()

# 세마포어 설정: 동시 요청 수 제한
semaphore = asyncio.Semaphore(2)

# 상태 코드에 따른 재시도 처리 헬퍼 함수
async def fetch_with_retry(session, url, headers, max_retries=5, retry_delay=2):
    for attempt in range(max_retries):
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                return await response.text()
            else:
                # print(f"Error {response.status}: Retrying ({attempt + 1}/{max_retries}) for URL: {url}")
                # 재시도 사이의 지연 시간
                await asyncio.sleep(retry_delay)
    # 모든 재시도 실패 시 None 반환
    print(f"Failed to fetch data from {url} after {max_retries} attempts.")
    return None

# 책 정보를 가져오는 함수
async def get_book_content(title, publisher):
    client_id = "id"
    client_secret = "secret"
    text = ' '.join([title, publisher])
    encText = urllib.parse.quote(text)
    url = f"https://openapi.naver.com/v1/search/book_adv.xml?d_titl={encText}&display=100"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    
    # 세마포어를 사용하여 동시 요청을 제어
    async with semaphore:
        async with aiohttp.ClientSession() as session:
            body = await fetch_with_retry(session, url, headers)
            if not body:  # 응답이 없을 경우 '응답없음' 반환
                return '응답없음'
            
            root = ET.fromstring(body)
            
            if root.find('channel').find('total').text == '0':
                return '없음'
            
            for item in root.find('channel').findall('item'):
                if item.find('publisher').text == publisher:
                    return item.find('description').text
            
            return '일치하지않음'

# 여러 책의 정보를 비동기로 가져오는 예시 함수
async def fetch_multiple_books(data):
    tasks = [get_book_content(d.get('title'), d.get('publisher')) for d in data]
    # tqdm_asyncio를 이용한 비동기 작업의 진행률 표시
    results = await tqdm_asyncio.gather(*tasks, desc="Fetching book data", total=len(data))
    for d, content in zip(data, results):
        if content in ['응답없음', '없음', '일치하지않음']:
            d['content'] = '없음'
        else:
            d['content'] = content
    return data


In [ ]:
# 파일 열기 및 데이터 로드
with open('./merged_data5.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 이벤트 루프 실행
result = asyncio.run(fetch_multiple_books(data))

# 파일 저장
with open('./processed_merged_data5.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=4)